<a href="https://colab.research.google.com/github/FAGomezValbuena/actividad-2-busqueda-sistemas-basados-en-reglas/blob/main/Sistema_Experto_Rutas_Cundinamarca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Instrucciones para ejecutar el sistema experto de rutas**

**1.1. Requisitos**
- Python 3.9 o superior
- pip instalado
- Librerías:
pip install networkx matplotlib ipywidgets pandas

**1.2. Opciones de ejecución**

    En Google Colab
- Abrir [https://colab.research.google.com](https://colab.research.google.com).
- Subir el archivo `Sistema_Experto_Rutas_Cundinamarca.ipynb`.
- Ejecutar las celdas en orden.

    En local (VS Code o terminal)
- Clonar el repositorio:
git clone https://github.com/TU_USUARIO/ Sistema_Experto_Rutas_Cundinamarca.git
cd rutas-intermunicipales-cundinamarca/src
- Ejecutar:
python Sistema_Experto_Rutas_Cundinamarca.py


**1.3. Uso**
- El programa permite seleccionar "origen" y "destino" entre 20 municipios de Cundinamarca.
- Calcula la mejor ruta considerando las reglas: a) Evitar peajes, b) Evitar vías en mal estado, c) Cierres viales y d) Muestra el grafo con colores y resalta la ruta óptima.


# **2. Instalar dependencias**

In [1]:
!pip install networkx pandas matplotlib ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.5 MB/s eta 0:00:00


# **3. Importar librerías**

In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


# ***4. Base de conocimiento y dataset de 20 municipios***

In [3]:
# Base de conocimiento (reglas lógicas)
reglas = {
    "evitar_peajes": False,               # Penaliza vías con peajes
    "evitar_vias_mal_estado": True,       # Penaliza tramos en mal estado
    "vias_cerradas": ["Fusagasuga-Silvania"]  # Tramos bloqueados
}

# Crear el grafo dirigido
G = nx.DiGraph()

# Dataset de rutas intermunicipales (origen, destino, tiempo en minutos, tipo_via)
rutas = [
    ("Bogotá","Soacha",30,"principal"),
    ("Bogotá","Chía",35,"principal"),
    ("Bogotá","Funza",25,"principal"),
    ("Bogotá","Fusagasuga",90,"peaje"),
    ("Bogotá","La Mesa",75,"peaje"),
    ("Bogotá","Zipaquira",60,"principal"),
    ("Soacha","Fusagasuga",70,"peaje"),
    ("Funza","Madrid",15,"principal"),
    ("Madrid","Facatativa",25,"principal"),
    ("Facatativa","La Mesa",60,"principal"),
    ("Chía","Cajica",15,"principal"),
    ("Cajica","Zipaquira",15,"principal"),
    ("Zipaquira","Nemocon",20,"principal"),
    ("Nemocon","Ubaté",50,"principal"),
    ("Ubaté","Choconta",60,"secundaria"),
    ("Choconta","Tocancipá",45,"secundaria"),
    ("Tocancipá","Sopó",15,"principal"),
    ("Sopó","Chía",20,"principal"),
    ("Fusagasuga","Silvania",20,"peaje"),
    ("Silvania","Girardot",70,"peaje"),
    ("La Mesa","Girardot",90,"peaje"),
]

# Agregar nodos y aristas al grafo
for origen, destino, tiempo, tipo in rutas:
    G.add_node(origen)
    G.add_node(destino)
    G.add_edge(origen, destino, weight=tiempo, tipo_via=tipo)
    G.add_edge(destino, origen, weight=tiempo, tipo_via=tipo)  # bidireccional


# **5. Motor de inferencia**

In [4]:
def aplicar_reglas(G, reglas):
    G_mod = G.copy()

    # Penalizar vías con peajes
    if reglas.get("evitar_peajes"):
        for u, v, data in G_mod.edges(data=True):
            if data.get("tipo_via") == "peaje":
                data["weight"] *= 1.3  # +30% al tiempo

    # Penalizar vías en mal estado (ejemplo: secundarias)
    if reglas.get("evitar_vias_mal_estado"):
        for u, v, data in G_mod.edges(data=True):
            if data.get("tipo_via") == "secundaria":
                data["weight"] *= 1.2  # +20% al tiempo

    # Eliminar rutas cerradas
    for tramo in reglas.get("vias_cerradas", []):
        partes = tramo.split("-")
        if len(partes) == 2:
            a, b = partes
            if G_mod.has_edge(a,b):
                G_mod.remove_edge(a,b)
            if G_mod.has_edge(b,a):
                G_mod.remove_edge(b,a)

    return G_mod


# **6. Algoritmo de búsqueda**

In [5]:
def mejor_ruta(G, origen, destino):
    try:
        ruta = nx.shortest_path(G, source=origen, target=destino, weight='weight')
        costo = nx.shortest_path_length(G, source=origen, target=destino, weight='weight')
        return ruta, costo
    except nx.NetworkXNoPath:
        return None, float('inf')


# **7. Visualización del grafo**

In [6]:
def mostrar_grafo(G, ruta=None):
    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(14,10))

    color_map = {
        "principal":"green",
        "secundaria":"orange",
        "peaje":"blue"
    }

    # Colores de aristas según tipo de vía
    edge_colors = [color_map.get(G[u][v]['tipo_via'], 'gray') for u,v in G.edges()]

    # Dibujar grafo
    nx.draw(G, pos,
            with_labels=True,
            node_color='lightyellow',
            node_size=800,
            edge_color=edge_colors,
            font_size=9)

    # Resaltar la ruta óptima
    if ruta:
        ruta_edges = list(zip(ruta[:-1], ruta[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=ruta_edges, edge_color='red', width=3)

    # Etiquetas de peso
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_size=8)

    plt.show()


# **8. Interfaz interactiva**

In [7]:
# Widgets para selección de reglas
check_peajes = widgets.Checkbox(
    value=False,
    description="Evitar peajes",
    indent=False
)

check_mal_estado = widgets.Checkbox(
    value=True,
    description="Evitar vías en mal estado",
    indent=False
)

check_cierres = widgets.Checkbox(
    value=False,
    description="Activar cierres viales",
    indent=False
)

# Menús desplegables para origen y destino
origen_dropdown = widgets.Dropdown(
    options=list(G.nodes),
    description="Origen:"
)

destino_dropdown = widgets.Dropdown(
    options=list(G.nodes),
    description="Destino:"
)

# Botones
boton_calcular = widgets.Button(
    description="Calcular mejor ruta",
    button_style='success'
)

boton_limpiar = widgets.Button(
    description="Limpiar búsqueda",
    button_style='warning'
)

# Área de salida
salida = widgets.Output()


# Función para aplicar reglas dinámicamente

def actualizar_reglas():
    return {
        "evitar_peajes": check_peajes.value,
        "evitar_vias_mal_estado": check_mal_estado.value,
        "vias_cerradas": ["Fusagasuga-Silvania"] if check_cierres.value else []
    }


# Evento al hacer clic en Calcular

def on_click_calcular(b):
    with salida:
        clear_output()  # Limpia resultados anteriores

        # Obtener valores de la interfaz
        origen = origen_dropdown.value
        destino = destino_dropdown.value

        # Aplicar reglas seleccionadas
        reglas_activas = actualizar_reglas()
        G_mod = aplicar_reglas(G, reglas_activas)

        # Calcular la mejor ruta
        ruta, costo = mejor_ruta(G_mod, origen, destino)

        if ruta:
            print(f"🚌 Mejor ruta: {' → '.join(ruta)}")
            print(f"⏱️ Tiempo estimado: {costo:.1f} minutos")
            mostrar_grafo(G_mod, ruta)
        else:
            print("❌ No existe una ruta disponible con las condiciones actuales.")


# Evento al hacer clic en Limpiar

def on_click_limpiar(b):
    with salida:
        clear_output()
        print("🔄 La interfaz ha sido restablecida. Selecciona nuevamente origen, destino y reglas.")

# Conectar eventos a los botones
boton_calcular.on_click(on_click_calcular)
boton_limpiar.on_click(on_click_limpiar)


# Mostrar interfaz

display(
    widgets.HBox([check_peajes, check_mal_estado, check_cierres]),
    origen_dropdown,
    destino_dropdown,
    widgets.HBox([boton_calcular, boton_limpiar]),
    salida
)


Dropdown(description='Origen:', options=('Bogotá', 'Soacha', 'Chía', 'Funza', 'Fusagasuga', 'La Mesa', 'Zipaqu…

Dropdown(description='Destino:', options=('Bogotá', 'Soacha', 'Chía', 'Funza', 'Fusagasuga', 'La Mesa', 'Zipaq…

Output()